In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
ind_chd = pd.read_csv('data/clean/ind-house-chd-clean-v1.csv')
df = ind_chd.copy()

In [3]:
df.head()

,property_id,property_name,property_type,link,Society,address1,address2,PricePerUnit,area,area_unit,...,OpenParking,RoadFacing,WaterSource,PowerBackup,WheelChairFriendly,PetFriendly,description,furnishDetails,features,price
0,R76627967,7Bedrooms 7Baths,House/Villa,https://www.99acres.com/7-bhk-bedroom-independ...,NaN,"Sector 15 Chandigarh, Chandigarh, India","Sector 15 Chandigarh, Chandigarh, India","1,40,000 per sq.yards",Plot area 2000(1672.25 sq.m.) sq.yards,sq.yards,...,3,40.0,"24*7 Water,Municipal corporation",NaN,Yes,Yes,This 7 bhk house for sale in sector 15 chandig...,"['4 Fan', '4 Light', 'No AC', 'No Bed', 'No Ch...","['Feng Shui / Vaastu Compliant', 'Water purifi...",28.000
1,V75627519,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,"Sector 11 Chandigarh, Chandigarh, India","Sector 11 Chandigarh, Chandigarh, India","2,80,000 per sq.yards",Plot area 500(418.06 sq.m.) sq.yards,sq.yards,...,1,40.0,Municipal corporation,NaN,Yes,No,Check out this 3 bhk house for sale in sector ...,NaN,"['False Ceiling Lighting', 'Separate entry for...",14.000
2,P75122867,6Bedrooms 6Baths,House/Villa,https://www.99acres.com/6-bhk-bedroom-independ...,Bajwa Sunny Urban Greens,"Bajwa Sunny Urban Greens, Mohali, Chandigarh, ...","Mohali, Chandigarh, India","6,944 per sq.ft.",Plot area 3600(334.45 sq.m.) sq.ft.,sq.ft.,...,0,NaN,NaN,NaN,No,No,We have a house/villa available for sale in mo...,[],NaN,2.500
3,W75661879,4Bedrooms 4Baths,House/Villa,https://www.99acres.com/4-bhk-bedroom-independ...,Altura Apartments,"Altura Apartments, Zirakpur, Chandigarh, India","Zirakpur, Chandigarh, India",813 per sq.ft.,Plot area 22000(2043.87 sq.m.) sq.ft.,sq.ft.,...,1,NaN,24*7 Water,Full,Yes,Yes,Penthouse with open bath tub very big terrace ...,"['6 Fan', '1 Fridge', '1 Exhaust Fan', '1 Dini...","['Feng Shui / Vaastu Compliant', 'Private Gard...",1.650
4,P75385469,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,"Dera Bassi, Chandigarh, India","Dera Bassi, Chandigarh, India","52,589 per sq.yards",Plot area 112(93.65 sq.m.) sq.yards,sq.yards,...,1,30.0,Borewell/Tank,NaN,No,No,Welcome to the epitome of comfortable living i...,"['16 Light', '1 Modular Kitchen', '1 Chimney',...","['Maintenance Staff', 'Water Storage', 'Park',...",0.589


In [4]:
pd.options.display.max_rows = None

In [5]:
df['address1']

0                 Sector 15 Chandigarh, Chandigarh, India
1                 Sector 11 Chandigarh, Chandigarh, India
2       Bajwa Sunny Urban Greens, Mohali, Chandigarh, ...
3          Altura Apartments, Zirakpur, Chandigarh, India
4                           Dera Bassi, Chandigarh, India
5                           Eco City 1, Chandigarh, India
6                  Sector 8 Chandigarh, Chandigarh, India
7       DLF Hyde Park Estate, Mullanpur, Chandigarh, I...
8                            Manimajra, Chandigarh, India
9                    Sigma City Colony, Chandigarh, India
10                       Maloya Colony, Chandigarh, India
11                          Dera Bassi, Chandigarh, India
12                               Daria, Chandigarh, India
13            42B Sector 42 Chandigarh, Chandigarh, India
14                Sector 46 Chandigarh, Chandigarh, India
15                Sector 44 Chandigarh, Chandigarh, India
16      Manohar Palm Residency, Mullanpur, Chandigarh,...
17            

In [6]:
df['address2']

0                 Sector 15 Chandigarh, Chandigarh, India
1                 Sector 11 Chandigarh, Chandigarh, India
2                               Mohali, Chandigarh, India
3                             Zirakpur, Chandigarh, India
4                           Dera Bassi, Chandigarh, India
5                           Eco City 1, Chandigarh, India
6             500, Sector 8 Chandigarh, Chandigarh, India
7                            Mullanpur, Chandigarh, India
8                            Manimajra, Chandigarh, India
9                    Sigma City Colony, Chandigarh, India
10                       Maloya Colony, Chandigarh, India
11                          Dera Bassi, Chandigarh, India
12                               Daria, Chandigarh, India
13            42B Sector 42 Chandigarh, Chandigarh, India
14                Sector 46 Chandigarh, Chandigarh, India
15                Sector 44 Chandigarh, Chandigarh, India
16                           Mullanpur, Chandigarh, India
17            

In [9]:
def address_func(x):
    if pd.notna(x):
        temp =  x.split(',')
        return temp[-3].strip(),temp[-2].strip()
    else:
        return x

In [10]:
df['address1'].apply(address_func)

0               (Sector 15 Chandigarh, Chandigarh)
1               (Sector 11 Chandigarh, Chandigarh)
2                             (Mohali, Chandigarh)
3                           (Zirakpur, Chandigarh)
4                         (Dera Bassi, Chandigarh)
5                         (Eco City 1, Chandigarh)
6                (Sector 8 Chandigarh, Chandigarh)
7                          (Mullanpur, Chandigarh)
8                          (Manimajra, Chandigarh)
9                  (Sigma City Colony, Chandigarh)
10                     (Maloya Colony, Chandigarh)
11                        (Dera Bassi, Chandigarh)
12                             (Daria, Chandigarh)
13          (42B Sector 42 Chandigarh, Chandigarh)
14              (Sector 46 Chandigarh, Chandigarh)
15              (Sector 44 Chandigarh, Chandigarh)
16                         (Mullanpur, Chandigarh)
17                          (Zirakpur, Chandigarh)
18                        (Dera Bassi, Chandigarh)
19                         (Mul

In [11]:
temp_df = df['address1'].apply(address_func).apply(pd.Series)
temp_df.columns = ['Sector','City']
temp_df

,Sector,City
0,Sector 15 Chandigarh,Chandigarh
1,Sector 11 Chandigarh,Chandigarh
2,Mohali,Chandigarh
3,Zirakpur,Chandigarh
4,Dera Bassi,Chandigarh
5,Eco City 1,Chandigarh
6,Sector 8 Chandigarh,Chandigarh
7,Mullanpur,Chandigarh
8,Manimajra,Chandigarh
9,Sigma City Colony,Chandigarh


In [12]:
df.insert(5,'City',value = temp_df['City'].values)
df.insert(5,'Sector',value = temp_df['Sector'].values)

In [13]:
df.head()

,property_id,property_name,property_type,link,Society,Sector,City,address1,address2,PricePerUnit,...,OpenParking,RoadFacing,WaterSource,PowerBackup,WheelChairFriendly,PetFriendly,description,furnishDetails,features,price
0,R76627967,7Bedrooms 7Baths,House/Villa,https://www.99acres.com/7-bhk-bedroom-independ...,NaN,Sector 15 Chandigarh,Chandigarh,"Sector 15 Chandigarh, Chandigarh, India","Sector 15 Chandigarh, Chandigarh, India","1,40,000 per sq.yards",...,3,40.0,"24*7 Water,Municipal corporation",NaN,Yes,Yes,This 7 bhk house for sale in sector 15 chandig...,"['4 Fan', '4 Light', 'No AC', 'No Bed', 'No Ch...","['Feng Shui / Vaastu Compliant', 'Water purifi...",28.000
1,V75627519,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,Sector 11 Chandigarh,Chandigarh,"Sector 11 Chandigarh, Chandigarh, India","Sector 11 Chandigarh, Chandigarh, India","2,80,000 per sq.yards",...,1,40.0,Municipal corporation,NaN,Yes,No,Check out this 3 bhk house for sale in sector ...,NaN,"['False Ceiling Lighting', 'Separate entry for...",14.000
2,P75122867,6Bedrooms 6Baths,House/Villa,https://www.99acres.com/6-bhk-bedroom-independ...,Bajwa Sunny Urban Greens,Mohali,Chandigarh,"Bajwa Sunny Urban Greens, Mohali, Chandigarh, ...","Mohali, Chandigarh, India","6,944 per sq.ft.",...,0,NaN,NaN,NaN,No,No,We have a house/villa available for sale in mo...,[],NaN,2.500
3,W75661879,4Bedrooms 4Baths,House/Villa,https://www.99acres.com/4-bhk-bedroom-independ...,Altura Apartments,Zirakpur,Chandigarh,"Altura Apartments, Zirakpur, Chandigarh, India","Zirakpur, Chandigarh, India",813 per sq.ft.,...,1,NaN,24*7 Water,Full,Yes,Yes,Penthouse with open bath tub very big terrace ...,"['6 Fan', '1 Fridge', '1 Exhaust Fan', '1 Dini...","['Feng Shui / Vaastu Compliant', 'Private Gard...",1.650
4,P75385469,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,Dera Bassi,Chandigarh,"Dera Bassi, Chandigarh, India","Dera Bassi, Chandigarh, India","52,589 per sq.yards",...,1,30.0,Borewell/Tank,NaN,No,No,Welcome to the epitome of comfortable living i...,"['16 Light', '1 Modular Kitchen', '1 Chimney',...","['Maintenance Staff', 'Water Storage', 'Park',...",0.589


## Sectors

In [14]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            76
Sector 35 Chandigarh            66
Zirakpur                        59
Sector 15 Chandigarh            55
Sector 38 Chandigarh            51
Sector 8 Chandigarh             48
Sector 34 Chandigarh            42
Sector 33 Chandigarh            41
Sector 18 Chandigarh            41
Sector 40 Chandigarh            40
Sector 27 Chandigarh            37
Sector 44 Chandigarh            36
Sector 19 Chandigarh            32
Sector 37 Chandigarh            31
Sector 16 Chandigarh            29
Dera Bassi                      28
Sector 22 Chandigarh            27
Mullanpur                       26
Sector 36 Chandigarh            26
Sector 46 Chandigarh            25
Sector 20 Chandigarh            25
Sector 32 Chandigarh            23
Sector 11 Chandigarh            22
Sector 28 Chandigarh            22
Sector 42 Chandigarh            20
Sector 43 Chandigarh            17
Sector 9 Chandigarh             16
Mohali                          16
Eco City 1   

In [15]:
def sector_clean(x):
    if pd.notna(x):
        if x.startswith('Sector'):
            output_str = re.sub(r'(\d+)([A-Za-z])', r'\1', x)
            return output_str
        elif x[0].isdigit():
            output_str = re.sub(r'\d+[A-Za-z]', '', x, count=1)
            return output_str.strip()
        else:
            return x
    else:
        return x

In [16]:
df['Sector'] = df['Sector'].apply(sector_clean)

In [17]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Sector 15 Chandigarh            60
Zirakpur                        59
Sector 38 Chandigarh            55
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 34 Chandigarh            46
Sector 33 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 20 Chandigarh            30
Sector 22 Chandigarh            30
Sector 16 Chandigarh            29
Dera Bassi                      28
Sector 36 Chandigarh            26
Mullanpur                       26
Sector 32 Chandigarh            26
Sector 46 Chandigarh            26
Sector 11 Chandigarh            22
Sector 28 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali                          16
Eco City 1   

In [18]:
## 'Sector 22 B Chandigarh', 'Sector 39 B Chandigarh','Sector 16 D Chandigarh'

In [19]:
temp_rows = df[df['Sector'] == 'Sector 22 B Chandigarh'].index
for i in temp_rows:
    df['Sector'].loc[i] = 'Sector 22 Chandigarh'

In [20]:
temp_rows = df[df['Sector'] == 'Sector 39 B Chandigarh'].index
for i in temp_rows:
    df['Sector'].loc[i] = 'Sector 39 Chandigarh'

In [21]:
temp_rows = df[df['Sector'] == 'Sector 16 D Chandigarh'].index
for i in temp_rows:
    df['Sector'].loc[i] = 'Sector 16 Chandigarh'

In [22]:
temp_rows = df[df['Sector'] == 'Sector 38 West'].index
for i in temp_rows:
    df['Sector'].loc[i] = 'Sector 38 Chandigarh'

In [23]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Sector 15 Chandigarh            60
Zirakpur                        59
Sector 38 Chandigarh            58
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 34 Chandigarh            46
Sector 33 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 22 Chandigarh            34
Sector 20 Chandigarh            30
Sector 16 Chandigarh            30
Dera Bassi                      28
Sector 36 Chandigarh            26
Sector 32 Chandigarh            26
Mullanpur                       26
Sector 46 Chandigarh            26
Sector 28 Chandigarh            22
Sector 11 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali                          16
Eco City 1   

#### Zirakpur,Mullanpur,Dera Bassi, Mohali

In [24]:
def sector_filt(sector):
    return df[df['Sector'] == sector][['Society','Sector','City']]

In [26]:
sector_filt('Zirakpur')

,Society,Sector,City
3,Altura Apartments,Zirakpur,Chandigarh
17,NaN,Zirakpur,Chandigarh
23,NaN,Zirakpur,Chandigarh
33,Premium Independent Floors,Zirakpur,Chandigarh
43,NaN,Zirakpur,Chandigarh
45,NaN,Zirakpur,Chandigarh
52,Highland Park,Zirakpur,Chandigarh
63,Metro Kings Court,Zirakpur,Chandigarh
68,NaN,Zirakpur,Chandigarh
69,Zirakpur Chandigarh,Zirakpur,Chandigarh


In [27]:
temp_rows = sector_filt('Zirakpur').index

In [28]:
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

In [29]:
sector_filt('Zirakpur')

,Society,Sector,City
3,Altura Apartments,Zirakpur,Mohali
17,NaN,Zirakpur,Mohali
23,NaN,Zirakpur,Mohali
33,Premium Independent Floors,Zirakpur,Mohali
43,NaN,Zirakpur,Mohali
45,NaN,Zirakpur,Mohali
52,Highland Park,Zirakpur,Mohali
63,Metro Kings Court,Zirakpur,Mohali
68,NaN,Zirakpur,Mohali
69,Zirakpur Chandigarh,Zirakpur,Mohali


In [30]:
sector_filt('Mullanpur')

,Society,Sector,City
7,DLF Hyde Park Estate,Mullanpur,Chandigarh
16,Manohar Palm Residency,Mullanpur,Chandigarh
19,Omaxe New Chandigarh,Mullanpur,Chandigarh
20,Omaxe New Chandigarh,Mullanpur,Chandigarh
29,DLF Hyde Park Estate,Mullanpur,Chandigarh
56,DLF Hyde Park Estate,Mullanpur,Chandigarh
66,Omaxe Silver Birch,Mullanpur,Chandigarh
144,NaN,Mullanpur,Chandigarh
145,Manohar Palm Residency,Mullanpur,Chandigarh
146,Omaxe Plots Phase 3,Mullanpur,Chandigarh


In [31]:
temp_rows = sector_filt('Mullanpur').index

In [32]:
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

In [33]:
sector_filt('Mullanpur')

,Society,Sector,City
7,DLF Hyde Park Estate,Mullanpur,Mohali
16,Manohar Palm Residency,Mullanpur,Mohali
19,Omaxe New Chandigarh,Mullanpur,Mohali
20,Omaxe New Chandigarh,Mullanpur,Mohali
29,DLF Hyde Park Estate,Mullanpur,Mohali
56,DLF Hyde Park Estate,Mullanpur,Mohali
66,Omaxe Silver Birch,Mullanpur,Mohali
144,NaN,Mullanpur,Mohali
145,Manohar Palm Residency,Mullanpur,Mohali
146,Omaxe Plots Phase 3,Mullanpur,Mohali


In [34]:
sector_filt('Dera Bassi')

,Society,Sector,City
4,NaN,Dera Bassi,Chandigarh
11,NaN,Dera Bassi,Chandigarh
18,Golden Palms,Dera Bassi,Chandigarh
24,Golden Palms,Dera Bassi,Chandigarh
27,GBP Rosewood Estate Phase 2,Dera Bassi,Chandigarh
35,NaN,Dera Bassi,Chandigarh
36,GBP ECO Greens 2,Dera Bassi,Chandigarh
40,Bella Homes,Dera Bassi,Chandigarh
1070,Ubber Garden Enclave,Dera Bassi,Chandigarh
1098,NaN,Dera Bassi,Chandigarh


In [35]:
temp_rows = sector_filt('Dera Bassi').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

#### Kharar, Badal Colony, Bhagat Singh Nagar, Lohgarh

In [36]:
sector_filt('Kharar')

,Society,Sector,City
1161,NaN,Kharar,Chandigarh
1170,NaN,Kharar,Chandigarh
1176,Paradise Darpan City,Kharar,Chandigarh
1191,NaN,Kharar,Chandigarh
1198,Paradise Darpan City,Kharar,Chandigarh


In [37]:
temp_rows = sector_filt('Kharar').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

In [38]:
sector_filt('Bhagat Singh Nagar')

,Society,Sector,City
1102,GBP Rosewood Estate Phase 2,Bhagat Singh Nagar,Chandigarh
1189,NaN,Bhagat Singh Nagar,Chandigarh


In [39]:
temp_rows = sector_filt('Bhagat Singh Nagar').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'
    df['Sector'].loc[i] = 'Dera Bassi'

In [40]:
sector_filt('Lohgarh')

,Society,Sector,City
21,NaN,Lohgarh,Chandigarh
32,NaN,Lohgarh,Chandigarh


In [41]:
temp_rows = sector_filt('Lohgarh').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'
    df['Sector'].loc[i] = 'Zirakpur'

#### Ecocity, New Chandigarh

In [42]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Zirakpur                        61
Sector 15 Chandigarh            60
Sector 38 Chandigarh            58
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 33 Chandigarh            46
Sector 34 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 22 Chandigarh            34
Sector 20 Chandigarh            30
Sector 16 Chandigarh            30
Dera Bassi                      30
Sector 46 Chandigarh            26
Sector 36 Chandigarh            26
Mullanpur                       26
Sector 32 Chandigarh            26
Sector 28 Chandigarh            22
Sector 11 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali                          16
Eco City 1   

In [43]:
sector_filt('Eco City 1')

,Society,Sector,City
5,NaN,Eco City 1,Chandigarh
22,NaN,Eco City 1,Chandigarh
48,NaN,Eco City 1,Chandigarh
58,NaN,Eco City 1,Chandigarh
154,NaN,Eco City 1,Chandigarh
365,NaN,Eco City 1,Chandigarh
404,NaN,Eco City 1,Chandigarh
475,NaN,Eco City 1,Chandigarh
476,NaN,Eco City 1,Chandigarh
477,NaN,Eco City 1,Chandigarh


In [44]:
temp_rows = sector_filt('Eco City 1').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'
    df['Sector'].loc[i] = 'New Chandigarh'
    df['Society'].loc[i] = 'Eco City'

In [45]:
sector_filt('New Chandigarh')

,Society,Sector,City
5,Eco City,New Chandigarh,Mohali
22,Eco City,New Chandigarh,Mohali
48,Eco City,New Chandigarh,Mohali
58,Eco City,New Chandigarh,Mohali
154,Eco City,New Chandigarh,Mohali
160,NaN,New Chandigarh,Chandigarh
180,NaN,New Chandigarh,Chandigarh
206,NaN,New Chandigarh,Chandigarh
207,NaN,New Chandigarh,Chandigarh
365,Eco City,New Chandigarh,Mohali


In [46]:
temp_rows = sector_filt('New Chandigarh').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

In [47]:
sector_filt('Mohali')

,Society,Sector,City
2,Bajwa Sunny Urban Greens,Mohali,Chandigarh
28,NaN,Mohali,Chandigarh
44,NaN,Mohali,Chandigarh
100,Bajwa Sunny Urban Greens,Mohali,Chandigarh
301,NaN,Mohali,Chandigarh
579,NaN,Mohali,Chandigarh
1101,NaN,Mohali,Chandigarh
1105,NaN,Mohali,Chandigarh
1153,NaN,Mohali,Chandigarh
1158,NaN,Mohali,Chandigarh


In [48]:
temp_rows = sector_filt('Mohali').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'

In [49]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Zirakpur                        61
Sector 15 Chandigarh            60
Sector 38 Chandigarh            58
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 33 Chandigarh            46
Sector 34 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 22 Chandigarh            34
Sector 20 Chandigarh            30
Sector 16 Chandigarh            30
Dera Bassi                      30
Sector 36 Chandigarh            26
Mullanpur                       26
Sector 32 Chandigarh            26
Sector 46 Chandigarh            26
New Chandigarh                  23
Sector 11 Chandigarh            22
Sector 28 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali       

In [50]:
#df.to_csv('data/clean/v2/chd/ind-house-chd-clean-v2.1.csv',index = False)

### OKOK

In [51]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Zirakpur                        61
Sector 15 Chandigarh            60
Sector 38 Chandigarh            58
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 33 Chandigarh            46
Sector 34 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 22 Chandigarh            34
Sector 20 Chandigarh            30
Sector 16 Chandigarh            30
Dera Bassi                      30
Sector 36 Chandigarh            26
Mullanpur                       26
Sector 32 Chandigarh            26
Sector 46 Chandigarh            26
New Chandigarh                  23
Sector 11 Chandigarh            22
Sector 28 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali       

In [52]:
# Burail -45

In [53]:
sector_filt('Sector45-a burail')

,Society,Sector,City
1181,NaN,Sector45-a burail,Chandigarh


In [54]:
sector_filt('Burail')

,Society,Sector,City
1187,NaN,Burail,Chandigarh


In [55]:
for i in [1181,1187]:
    df['Sector'].loc[i] = 'Sector 45 Chandigarh'
    df['Society'].loc[i] = 'Burail'

In [56]:
# Badal Colony - Zirakpur, Saidpura - Dera Bassi 

In [53]:
sector_filt('Badal Colony')

,Society,Sector,City
26,NK SHARMA ROAD,Badal Colony,Chandigarh
67,,Badal Colony,Chandigarh
1138,,Badal Colony,Chandigarh


In [57]:
temp_rows = sector_filt('Badal Colony').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'
    df['Sector'].loc[i] = 'Zirakpur'

In [58]:
sector_filt('Saidpura')

,Society,Sector,City
1103,NaN,Saidpura,Chandigarh


In [59]:
df['City'].loc[1103] = 'Mohali'
df['Sector'].loc[1103] = 'Dera Bassi'

In [60]:
# Aerocity - Mohali, 
sector_filt('Aerocity')

,Society,Sector,City
30,NaN,Aerocity,Chandigarh


In [61]:
df['City'].loc[30] = 'Mohali'

In [62]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh            87
Sector 35 Chandigarh            72
Zirakpur                        64
Sector 15 Chandigarh            60
Sector 38 Chandigarh            58
Sector 8 Chandigarh             49
Sector 18 Chandigarh            47
Sector 34 Chandigarh            46
Sector 33 Chandigarh            46
Sector 40 Chandigarh            45
Sector 27 Chandigarh            40
Sector 44 Chandigarh            39
Sector 37 Chandigarh            36
Sector 19 Chandigarh            35
Sector 22 Chandigarh            34
Dera Bassi                      31
Sector 16 Chandigarh            30
Sector 20 Chandigarh            30
Sector 36 Chandigarh            26
Sector 46 Chandigarh            26
Mullanpur                       26
Sector 32 Chandigarh            26
New Chandigarh                  23
Sector 28 Chandigarh            22
Sector 11 Chandigarh            22
Sector 42 Chandigarh            21
Sector 43 Chandigarh            19
Sector 9 Chandigarh             17
Mohali       

In [63]:
# Shivalik Vihar 
sector_filt('Shivalik Vihar')

,Society,Sector,City
1154,NaN,Shivalik Vihar,Chandigarh


In [64]:
df['Sector'].loc[1154] = 'Naya Gaon'

In [65]:
sector_filt('Panchkula')

,Society,Sector,City
1065,NaN,Panchkula,Chandigarh


In [66]:
df['link'].loc[1065]

'https://www.99acres.com/2-bhk-bedroom-independent-house-villa-for-sale-in-chandigarh-1575-sq-ft-spid-Y76446577'

In [67]:
df.drop(1065,inplace = True)

In [70]:
df['Sector'].value_counts().tail(25)

Sector
Vip Road                        1
Shivalik Enclave                1
Govind Nagar                    1
Sector 49 Chandigarh            1
Ashiana Colony                  1
Kishangarh                      1
Sector 25 Chandigarh            1
Sector 5 Chandigarh             1
Sector 39 Chandigarh            1
Sector 63 Chandigarh            1
Independent house Villa         1
Hiland Villa                    1
Independent House Villa         1
Riverdale Aerovista Zirakpur    1
Manohar New Chandigarh          1
High ground road                1
Sigma City Colony               1
Behlana                         1
Khuda Ali Sher                  1
Aerocity                        1
sukhna enclave block c          1
Dhanas                          1
pipliwala town                  1
Sector 29 Chandigarh            1
Sector 1 Chandigarh             1
Name: count, dtype: int64

In [69]:
df['Sector'].value_counts().tail(25).index

Index(['Vip Road', 'Shivalik Enclave', 'Govind Nagar', 'Sector 49 Chandigarh',
       'Ashiana Colony', 'Kishangarh', 'Sector 25 Chandigarh',
       'Sector 5 Chandigarh', 'Sector 39 Chandigarh', 'Sector 63 Chandigarh',
       'Independent house Villa', 'Hiland Villa', 'Independent House Villa',
       'Riverdale Aerovista Zirakpur', 'Manohar New Chandigarh',
       'High ground road', 'Sigma City Colony', 'Behlana', 'Khuda Ali Sher',
       'Aerocity', 'sukhna enclave block c', 'Dhanas', 'pipliwala town',
       'Sector 29 Chandigarh', 'Sector 1 Chandigarh'],
      dtype='object', name='Sector')

In [71]:
sect_to_remove = list(df['Sector'].value_counts().tail(25).index)

In [72]:
for i in ['Sector 49 Chandigarh','Sector 25 Chandigarh','Sector 39 Chandigarh','Sector 5 Chandigarh',
          'Aerocity','Dhanas','Sector 29 Chandigarh','Sector 63 Chandigarh']:
    sect_to_remove.remove(i)

In [73]:
sect_to_remove

['Vip Road',
 'Shivalik Enclave',
 'Govind Nagar',
 'Ashiana Colony',
 'Kishangarh',
 'Independent house Villa',
 'Hiland Villa',
 'Independent House Villa',
 'Riverdale Aerovista Zirakpur',
 'Manohar New Chandigarh',
 'High ground road',
 'Sigma City Colony',
 'Behlana',
 'Khuda Ali Sher',
 'sukhna enclave block c',
 'pipliwala town',
 'Sector 1 Chandigarh']

In [74]:
drop_rows = []
for i in sect_to_remove:
    drop_rows += list(sector_filt(i).index)

In [75]:
drop_rows

[581,
 1130,
 1164,
 1182,
 1094,
 263,
 262,
 261,
 179,
 171,
 156,
 9,
 102,
 25,
 31,
 39,
 1200]

In [76]:
df.drop(drop_rows,inplace = True)

In [77]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh    87
Sector 35 Chandigarh    72
Zirakpur                64
Sector 15 Chandigarh    60
Sector 38 Chandigarh    58
Sector 8 Chandigarh     49
Sector 18 Chandigarh    47
Sector 33 Chandigarh    46
Sector 34 Chandigarh    46
Sector 40 Chandigarh    45
Sector 27 Chandigarh    40
Sector 44 Chandigarh    39
Sector 37 Chandigarh    36
Sector 19 Chandigarh    35
Sector 22 Chandigarh    34
Dera Bassi              31
Sector 20 Chandigarh    30
Sector 16 Chandigarh    30
Sector 36 Chandigarh    26
Sector 46 Chandigarh    26
Sector 32 Chandigarh    26
Mullanpur               26
New Chandigarh          23
Sector 11 Chandigarh    22
Sector 28 Chandigarh    22
Sector 42 Chandigarh    21
Sector 43 Chandigarh    19
Sector 9 Chandigarh     17
Mohali                  16
Sector 10 Chandigarh    11
Sector 23 Chandigarh    11
Manimajra               11
Naya Gaon                9
Sector 7 Chandigarh      8
Sector 2 Chandigarh      7
Kharar                   5
Daria                

In [78]:
sector_filt('Sector 63 Chandigarh')

,Society,Sector,City
59,NaN,Sector 63 Chandigarh,Chandigarh


In [79]:
df['link'].loc[59]

'https://www.99acres.com/1-bhk-bedroom-independent-house-villa-for-sale-in-sector-63-chandigarh-750-sq-ft-spid-V75988367'

In [80]:
temp_rows = sector_filt('Sector 63 Chandigarh').index
for i in temp_rows:
    df['City'].loc[i] = 'Mohali'
    df['Sector'].loc[i] = 'Sector 63 Mohali'

In [81]:
df['Sector'].value_counts()

Sector
Sector 21 Chandigarh    87
Sector 35 Chandigarh    72
Zirakpur                64
Sector 15 Chandigarh    60
Sector 38 Chandigarh    58
Sector 8 Chandigarh     49
Sector 18 Chandigarh    47
Sector 33 Chandigarh    46
Sector 34 Chandigarh    46
Sector 40 Chandigarh    45
Sector 27 Chandigarh    40
Sector 44 Chandigarh    39
Sector 37 Chandigarh    36
Sector 19 Chandigarh    35
Sector 22 Chandigarh    34
Dera Bassi              31
Sector 20 Chandigarh    30
Sector 16 Chandigarh    30
Sector 36 Chandigarh    26
Sector 46 Chandigarh    26
Sector 32 Chandigarh    26
Mullanpur               26
New Chandigarh          23
Sector 11 Chandigarh    22
Sector 28 Chandigarh    22
Sector 42 Chandigarh    21
Sector 43 Chandigarh    19
Sector 9 Chandigarh     17
Mohali                  16
Sector 10 Chandigarh    11
Sector 23 Chandigarh    11
Manimajra               11
Naya Gaon                9
Sector 7 Chandigarh      8
Sector 2 Chandigarh      7
Kharar                   5
Daria                

### Society

In [82]:
df['Society'].value_counts()

Society
Eco City                       18
DLF Hyde Park Estate            9
Modern Housing Complex          7
Zirakpur Chandigarh             7
Omaxe New Chandigarh            5
Golden Palms                    4
Ubber Garden Enclave            3
Bajwa Sunny Urban Greens        3
Premium Independent Floors      2
Burail                          2
Paradise Darpan City            2
GBP Rosewood Estate Phase 2     2
Manohar Palm Residency          2
Victoria Enclave                2
Satyam Swastik Vihar            2
Chandigarh Royale City          2
Altus Muirwood Ecocity          1
Ubber Golden Palm Premium       1
GBP Astra                       1
GBP Rosewood Estate             1
VIP Enclave                     1
Pride Homes                     1
Ubber Palm City                 1
Central Town                    1
Altus Muirwoods                 1
Omaxe Mulberry                  1
Silver City Extention           1
Mittal Friends Enclave          1
Penta Vip Towers                1
Altura

In [84]:
def society_filt(society):
    return df[df['Society'] == society][['Society','Sector','City']]

In [85]:
# Eco city
society_filt('Eco City 1')

,Society,Sector,City


In [86]:
society_filt('Eco City')

,Society,Sector,City
5,Eco City,New Chandigarh,Mohali
22,Eco City,New Chandigarh,Mohali
48,Eco City,New Chandigarh,Mohali
58,Eco City,New Chandigarh,Mohali
148,Eco City,Mullanpur,Mohali
154,Eco City,New Chandigarh,Mohali
264,Eco City,Mullanpur,Mohali
365,Eco City,New Chandigarh,Mohali
404,Eco City,New Chandigarh,Mohali
475,Eco City,New Chandigarh,Mohali


In [87]:
temp_rows = society_filt('Eco City').index
for i in temp_rows:
    df['Sector'].loc[i] = 'New Chandigarh'

In [88]:
df['Society'].value_counts()

Society
Eco City                       18
DLF Hyde Park Estate            9
Modern Housing Complex          7
Zirakpur Chandigarh             7
Omaxe New Chandigarh            5
Golden Palms                    4
Ubber Garden Enclave            3
Bajwa Sunny Urban Greens        3
Premium Independent Floors      2
Burail                          2
Paradise Darpan City            2
GBP Rosewood Estate Phase 2     2
Manohar Palm Residency          2
Victoria Enclave                2
Satyam Swastik Vihar            2
Chandigarh Royale City          2
Altus Muirwood Ecocity          1
Ubber Golden Palm Premium       1
GBP Astra                       1
GBP Rosewood Estate             1
VIP Enclave                     1
Pride Homes                     1
Ubber Palm City                 1
Central Town                    1
Altus Muirwoods                 1
Omaxe Mulberry                  1
Silver City Extention           1
Mittal Friends Enclave          1
Penta Vip Towers                1
Altura

In [90]:
df.head()

,property_id,property_name,property_type,link,Society,Sector,City,address1,address2,PricePerUnit,...,OpenParking,RoadFacing,WaterSource,PowerBackup,WheelChairFriendly,PetFriendly,description,furnishDetails,features,price
0,R76627967,7Bedrooms 7Baths,House/Villa,https://www.99acres.com/7-bhk-bedroom-independ...,NaN,Sector 15 Chandigarh,Chandigarh,"Sector 15 Chandigarh, Chandigarh, India","Sector 15 Chandigarh, Chandigarh, India","1,40,000 per sq.yards",...,3,40.0,"24*7 Water,Municipal corporation",NaN,Yes,Yes,This 7 bhk house for sale in sector 15 chandig...,"['4 Fan', '4 Light', 'No AC', 'No Bed', 'No Ch...","['Feng Shui / Vaastu Compliant', 'Water purifi...",28.000
1,V75627519,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,Sector 11 Chandigarh,Chandigarh,"Sector 11 Chandigarh, Chandigarh, India","Sector 11 Chandigarh, Chandigarh, India","2,80,000 per sq.yards",...,1,40.0,Municipal corporation,NaN,Yes,No,Check out this 3 bhk house for sale in sector ...,NaN,"['False Ceiling Lighting', 'Separate entry for...",14.000
2,P75122867,6Bedrooms 6Baths,House/Villa,https://www.99acres.com/6-bhk-bedroom-independ...,Bajwa Sunny Urban Greens,Mohali,Mohali,"Bajwa Sunny Urban Greens, Mohali, Chandigarh, ...","Mohali, Chandigarh, India","6,944 per sq.ft.",...,0,NaN,NaN,NaN,No,No,We have a house/villa available for sale in mo...,[],NaN,2.500
3,W75661879,4Bedrooms 4Baths,House/Villa,https://www.99acres.com/4-bhk-bedroom-independ...,Altura Apartments,Zirakpur,Mohali,"Altura Apartments, Zirakpur, Chandigarh, India","Zirakpur, Chandigarh, India",813 per sq.ft.,...,1,NaN,24*7 Water,Full,Yes,Yes,Penthouse with open bath tub very big terrace ...,"['6 Fan', '1 Fridge', '1 Exhaust Fan', '1 Dini...","['Feng Shui / Vaastu Compliant', 'Private Gard...",1.650
4,P75385469,3Bedrooms 3Baths,House/Villa,https://www.99acres.com/3-bhk-bedroom-independ...,NaN,Dera Bassi,Mohali,"Dera Bassi, Chandigarh, India","Dera Bassi, Chandigarh, India","52,589 per sq.yards",...,1,30.0,Borewell/Tank,NaN,No,No,Welcome to the epitome of comfortable living i...,"['16 Light', '1 Modular Kitchen', '1 Chimney',...","['Maintenance Staff', 'Water Storage', 'Park',...",0.589


In [91]:
df.to_csv('data/clean/v2/chd/ind-house-chd-with-extra-address.csv',index = False)

In [92]:
df.drop(columns=['address1','address2'],inplace = True)

In [93]:
df.to_csv('data/clean/v2/chd/ind-house-chd-clean-v2.csv',index = False)